In [ ]:
"""
04/28/2024 00:00 systron PSL (SN: 999)
0000009 0.002179841,-0.004321344,-0.007393281,0.1804913,-0.09683202,-3.639365,1
0000119 -0.002570751,-0.005,-0.002642688,0.01625653,-0.07443636,-3.662439,1
0000213 -0.002570751,-0.005,-0.0101079,0.008791309,-0.0181079,-3.700444,1
0000305 -0.01071462,-0.006357312,-0.005357312,-0.04889446,0.05382964,-3.749306,1
"""